In [1]:
import os

In [2]:
%pwd

'c:\\Users\\rahul\\Desktop\\Project\\Machine-Learning-using-Python\\05. XG Boost'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\rahul\\Desktop\\Project\\Machine-Learning-using-Python'

## Problem Statement

Rossmann operates over 3000 drug stores in 7 european countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to 6 weeks in advance. Store Sales are influenced by many factors, including promotions, competitions, school holidays, seasonality and locality.

With thousand of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied. You are provided with historical sales for 1150 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

##### Downloading the dataset

In [5]:
import opendatasets as od
import pandas as pd

pd.set_option("display.max_columns", 120)
pd.set_option("display.min_rows", 120)

In [6]:
od.download("https://www.kaggle.com/c/rossmann-store-sales", data_dir = "Dataset/")

Skipping, found downloaded files in "Dataset/rossmann-store-sales" (use force=True to force download)


In [7]:
os.listdir("Dataset/rossmann-store-sales")

['sample_submission.csv', 'store.csv', 'test.csv', 'train.csv']

In [8]:
ross_df = pd.read_csv("Dataset/rossmann-store-sales/train.csv", low_memory = False)
store_df = pd.read_csv("Dataset/rossmann-store-sales/store.csv")
test_df = pd.read_csv("Dataset/rossmann-store-sales/test.csv")
submission_df = pd.read_csv("Dataset/rossmann-store-sales/sample_submission.csv")

In [9]:
ross_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [10]:
store_df.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [11]:
test_df.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [12]:
submission_df.head()

,Id,Sales
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


Let's merge the information from the store_df into train_df and test_df

In [13]:
merged_df = ross_df.merge(store_df, how = "left", on = "Store")
merged_test_df = test_df.merge(store_df, how = "left", on = "Store")

In [14]:
merged_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


#### Preprocess and Feature Engineering 

In [15]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Date                       1017209 non-null  object 
 3   Sales                      1017209 non-null  int64  
 4   Customers                  1017209 non-null  int64  
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   StoreType                  1017209 non-null  object 
 10  Assortment                 1017209 non-null  object 
 11  CompetitionDistance        1014567 non-null  float64
 12  CompetitionOpenSinceMonth  693861 non-null   float64
 13  CompetitionO

##### Date 

In [16]:
def split_date(df):
    df["Date"] = pd.to_datetime(df["Date"])
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    df["WeekOfYear"] = df["Date"].dt.isocalendar().week

In [17]:
split_date(merged_df)
split_date(merged_test_df)

In [18]:
merged_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day,WeekOfYear
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015,7,31,31
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2015,7,31,31
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",2015,7,31,31
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN,2015,7,31,31
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN,2015,7,31,31


##### Store Open/Close

In [19]:
merged_df[merged_df["Open"] == 0]["Sales"].value_counts()

Sales
0    172817
Name: count, dtype: int64

In [20]:
merged_df = merged_df[merged_df["Open"] == 1].copy()

##### Competition

In [21]:
def comp_months(df):
    df["CompetitionOpen"] = 12 * (df.Year - df.CompetitionOpenSinceYear) + (df.Month - df.CompetitionOpenSinceMonth)
    df["CompetitionOpen"] = df["CompetitionOpen"].map(lambda x: 0 if x < 0 else x).fillna(0)

In [22]:
comp_months(merged_df)
comp_months(merged_test_df)

In [23]:
merged_df[["Date", "CompetitionDistance", "CompetitionOpenSinceYear", "CompetitionOpenSinceMonth", "CompetitionOpen"]].sample(10)

,Date,CompetitionDistance,CompetitionOpenSinceYear,CompetitionOpenSinceMonth,CompetitionOpen
310021,2014-10-14,14040.0,NaN,NaN,0.0
446875,2014-05-27,580.0,2005.0,1.0,112.0
715483,2013-09-28,1350.0,2009.0,9.0,48.0
266338,2014-11-29,9710.0,2014.0,2.0,9.0
765299,2013-08-14,7910.0,NaN,NaN,0.0
611480,2013-12-30,920.0,2015.0,7.0,0.0
719680,2013-09-24,3770.0,NaN,NaN,0.0
853116,2013-05-28,480.0,NaN,NaN,0.0
339769,2014-09-12,2130.0,2008.0,12.0,69.0
80341,2015-05-20,2050.0,NaN,NaN,0.0


In [24]:
def check_promo_month(row):
    month2str = {1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun",
                 7: "Jul", 8: "Aug", 9: "Sept", 10: "Oct", 11: "Nov", 12: "Dec"}
    
    try:
        months = (row["PromoInterval"] or "").split(",")
        if row["Promo2Open"] and month2str[row["Month"]] in months:
            return 1
        else:
            return 0
    except Exception:
        return 0
    
def promo_cols(df):
    df["Promo2Open"] = (df.Year - df.Promo2SinceYear) * 12 + (df.WeekOfYear - df.Promo2SinceWeek) / 4.345
    df["Promo2Open"] = df["Promo2Open"].map(lambda x: 0 if x < 0 else x).fillna(0) * df["Promo2"]
    
    df["IsPromo2Month"] = df.apply(check_promo_month, axis = 1) * df["Promo2"]

In [25]:
promo_cols(merged_df)
promo_cols(merged_test_df)

In [26]:
merged_df[["Date", "Promo2", "Promo2SinceYear", "Promo2SinceWeek", "PromoInterval", "Promo2Open", "IsPromo2Month"]].sample(5)

,Date,Promo2,Promo2SinceYear,Promo2SinceWeek,PromoInterval,Promo2Open,IsPromo2Month
695119,2013-10-16,1,2009.0,45.0,"Feb,May,Aug,Nov",47.309551,0
664592,2013-11-13,1,2011.0,48.0,"Mar,Jun,Sept,Dec",23.539701,0
413177,2014-06-26,0,NaN,NaN,NaN,0.000000,0
271423,2014-11-24,0,NaN,NaN,NaN,0.000000,0
646653,2013-11-29,1,2011.0,22.0,"Jan,Apr,Jul,Oct",29.983890,0


In [27]:
merged_df.shape

(844392, 25)

#### Inputs and Target Columns

In [28]:
merged_df.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'Year', 'Month', 'Day',
       'WeekOfYear', 'CompetitionOpen', 'Promo2Open', 'IsPromo2Month'],
      dtype='object')

In [29]:
input_cols = ["Store", "DayOfWeek", "Promo", "StateHoliday", "SchoolHoliday",
              "StoreType", "Assortment", "CompetitionDistance", "CompetitionOpen",
              "Day", "Month", "Year", "WeekOfYear", "Promo2", "Promo2Open", "IsPromo2Month"]

target_col = "Sales"

In [30]:
inputs = merged_df[input_cols].copy()
targets = merged_df[target_col].copy()

In [31]:
test_inputs = merged_test_df[input_cols].copy()

In [32]:
inputs.head()

,Store,DayOfWeek,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpen,Day,Month,Year,WeekOfYear,Promo2,Promo2Open,IsPromo2Month
0,1,5,1,0,1,c,a,1270.0,82.0,31,7,2015,31,0,0.000000,0
1,2,5,1,0,1,a,a,570.0,92.0,31,7,2015,31,1,64.142693,1
2,3,5,1,0,1,a,a,14130.0,103.0,31,7,2015,31,1,51.912543,1
3,4,5,1,0,1,c,c,620.0,70.0,31,7,2015,31,0,0.000000,0
4,5,5,1,0,1,a,a,29910.0,3.0,31,7,2015,31,0,0.000000,0


In [33]:
numeric_cols = ["Store", "Promo", "SchoolHoliday",
                "CompetitionDistance", "CompetitionOpen", "Promo2", "Promo2Open", "IsPromo2Month",
                "Day", "Month", "Year", "WeekOfYear"]

cat_cols = [col for col in input_cols if col not in numeric_cols]

#### Impute missing numerical data

In [34]:
inputs[numeric_cols].isna().sum()

Store                     0
Promo                     0
SchoolHoliday             0
CompetitionDistance    2186
CompetitionOpen           0
Promo2                    0
Promo2Open                0
IsPromo2Month             0
Day                       0
Month                     0
Year                      0
WeekOfYear                0
dtype: int64

In [35]:
test_inputs[numeric_cols].isna().sum()

Store                   0
Promo                   0
SchoolHoliday           0
CompetitionDistance    96
CompetitionOpen         0
Promo2                  0
Promo2Open              0
IsPromo2Month           0
Day                     0
Month                   0
Year                    0
WeekOfYear              0
dtype: int64

In [36]:
max_distance = inputs.CompetitionDistance.max()

In [37]:
inputs["CompetitionDistance"] = inputs["CompetitionDistance"].fillna(max_distance * 2)
test_inputs["CompetitionDistance"] = test_inputs["CompetitionDistance"].fillna(max_distance * 2)

#### Scaling the numerical values

In [38]:
from sklearn.preprocessing import MinMaxScaler

In [39]:
scaler = MinMaxScaler().fit(inputs[numeric_cols])

In [40]:
inputs[numeric_cols] = scaler.transform(inputs[numeric_cols])
test_inputs[numeric_cols] = scaler.transform(test_inputs[numeric_cols])

#### Encode Categorical Columns

In [41]:
from sklearn.preprocessing import OneHotEncoder

In [42]:
encoder = OneHotEncoder(sparse_output = False, handle_unknown = "ignore").fit(inputs[cat_cols])
encoded_columns = list(encoder.get_feature_names_out(cat_cols))

In [43]:
inputs[encoded_columns] = encoder.transform(inputs[cat_cols])
test_inputs[encoded_columns] = encoder.transform(test_inputs[cat_cols])

In [44]:
X_train = inputs[numeric_cols + encoded_columns]
X_test = test_inputs[numeric_cols + encoded_columns]

We haven't created a validation set because we will be using k-fold cross validation.